In [1]:
import sys
import os
%pwd
# sys.path

'f:\\Data Scientist\\TextSummarizerProject\\research'

In [2]:
os.chdir("../")
%pwd

'f:\\Data Scientist\\TextSummarizerProject'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir:Path

In [6]:
# %load_ext autoreload

# %autoreload 2

from textSummarizer.constants.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH, MY_CONSTANT
# from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_file_path
# print(os.getcwd())
# import importlib
# importlib.reload(textSummarizer) 
# importlib.reload(textSummarizer.utils.common)
# MY_CONSTANT


In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):       
        # get_file_path()
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        return data_ingestion_config
# test = ConfigurationManager()
    

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from urllib.error import HTTPError, URLError


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                # print(self.config.source_URL)
                filename,headers = request.urlretrieve(url = self.config.source_URL, filename=self.config.local_data_file)
                logger.info(f"{filename} download! with following info: \n{headers}")
            else:
                logger.info(f"File {Path(self.config.local_data_file)} already exists of size {get_size(Path(self.config.local_data_file))}")
        except HTTPError as e:
            if e.code == 404:
                print(f"HTTP Error 404: The requested resource at {url} was not found.")
            else:
                print(f"HTTP Error {e.code}: {e.reason} for URL: {url}")
        except URLError as e:
            print(f"URL Error: {e.reason} for URL: {url}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted all the files to {unzip_path}")
    def download_extract_zip_file(self):
        zip_path, _ = request.urlretrieve(self.config.source_URL)
        logger.info(f"{zip_path} download! with following info:")
        with zipfile.ZipFile(zip_path, "r") as f:
            f.extractall(self.config.unzip_dir)   

            

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    # data_ingestion.download_extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e


[2025-06-26 21:29:15,312:INFO:common:yaml file: config\config.yaml loaded successfully.]


[2025-06-26 21:29:15,340:INFO:common:yaml file: params.yaml loaded successfully.]
[2025-06-26 21:29:15,341:INFO:common:created directory at: artifacts]
[2025-06-26 21:29:15,345:INFO:common:created directory at: artifacts/data_ingestion]
[2025-06-26 21:29:15,349:INFO:3985408789:File artifacts\data_ingestion_data.zip already exists of size ~ 23073 KB]
[2025-06-26 21:29:16,557:INFO:3985408789:Extracted all the files to artifacts\data_ingestion]
